In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import scrapping_with_selenium as sc  # scrapping_with_selenium: past project which scrapes data from wuzzuf(check it in my past repo)

In [ ]:
# the function which scrapes the data
df = sc.scraper("Data", 2, ["Data Engineer","Analyst", "Data Scientist", "Bi", "Business Intelligence","Data Entry"])

In [ ]:
df.to_csv("D:\Projects\Data Roles Market\df.csv")